## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Summarization

Text summarization involves condensing lengthy textual content into a brief format while retaining its essential information and significance. The primary aim is to extract key details from a text document and present them concisely and comprehensibly. It plays a crucial role across various domains, including healthcare, aiding in efficient communication and decision-making.


- **Model**: `en.summarize.clinical_laymen_onnx.pipeline`
- **Model Description**: This pretrained pipeline is built on the top of summarizer_clinical_laymen_onnx model, which is is a modified version of LLM based summarization model that is finetuned with custom dataset by John Snow Labs to avoid using clinical jargon on the summaries. It can generate summaries up to 512 tokens given an input text (max 1024 tokens).

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name = "en-summarize-clinical-laymen-onnx-pipeline"

real_time_inference_instance_type = "ml.m4.2xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

### A. Create an endpoint

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-------!

Once endpoint has been created, you would be able to perform real-time inference.

### B. Perform real-time inference

In [6]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial setup

In [7]:
docs = [

    """Samantha Brown came to my clinic for a consultation regarding the option of bariatric surgery for weight loss on November 12, 2010. Samantha Brown, a 38-year-old woman with a BMI of 46, is 5'7" tall and weighs 293 pounds. The reason she is considering surgical intervention for weight loss is her ongoing struggle with obesity for more than 25 years and her desire to enhance her physical stamina and self-perception. Not only does her weight cause physical limitations, it also hampers her social interactions. Every time she has lost weight, she has ended up regaining it, often adding more pounds than she initially lost. For instance, she once shed 120 pounds, only to gain it back within a year. She has attempted various commercial diets and weight loss programs, such as Jenny Craig for five months in 1993 with a 20-pound loss, Optifast for three months in 1991 with a seven-pound loss, Herbalife for seven weeks in 2006 with a nine-pound loss, a gym regimen for three months in 2008 with a six-pound loss, South Beach Diet for four months in 2009 with a twelve-pound loss, and Alli for two months in 2006 with a six-pound loss. She has also experimented with numerous low-fat or fad diets. She took Fen-Phen for ten months with a 120-pound loss.
    PAST MEDICAL HISTORY: She has a history of diabetes and sleep apnea.


    PAST SURGICAL HISTORY: Noteworthy for appendectomy.


    PSYCHOLOGICAL HISTORY: Unremarkable.


    SOCIAL HISTORY: She is divorced. She has an occasional glass of wine. She has never smoked.


    FAMILY HISTORY: Significant for obesity and type 2 diabetes.


    MEDICATIONS: Metformin 500 mg twice daily, Amlodipine 5 mg daily, Lipitor 20 mg daily, Metoprolol 50 mg twice daily, and a daily multivitamin.


    ALLERGIES: No known drug allergies.


    REVIEW OF SYSTEMS: Unremarkable.


    PHYSICAL EXAM: Samantha is an amiable woman appearing comfortable. Alert and fully oriented. HEENT: Normocephalic, without trauma. Extraocular movements are intact, sclerae are non-icteric. Lungs are clear upon bilateral auscultation. Cardiovascular system shows a regular rhythm. Abdomen is obese, pliable, without tenderness or distension. Extremities exhibit no signs of edema, clubbing, or cyanosis.


    ASSESSMENT/PLAN: Samantha Brown is a 38-year-old woman with a BMI of 46, who is contemplating bariatric surgery, specifically gastric bypass rather than the Lap-Band. Samantha Brown will be requesting a letter of medical necessity from Dr. Benjamin Harris. She will also schedule appointments with my nutritionist and social worker, and will undergo an upper endoscopy. Once these steps are done, we will forward her application to her insurance company for approval.""",


    """Patient Name: John Davis
    Age: 45


    Gender: Male


    Diagnosis: Acute Appendicitis


    Procedure: Appendectomy


    Summary: Mr. Davis presented with severe abdominal pain localized to the right lower quadrant. Physical examination and imaging confirmed the diagnosis of acute appendicitis. The patient underwent an emergency appendectomy, and the procedure was uneventful. Postoperatively, the patient recovered well and was discharged after two days.""",

]



sample_text = """Olivia Smith was seen in my office for evaluation for elective surgical weight loss on October 6, 2008. Olivia Smith is a 34-year-old female with a BMI of 43. She is 5'6" tall and weighs 267 pounds. She is motivated to attempt surgical weight loss because she has been overweight for over 20 years and wants to have more energy and improve her self-image. She is not only affected physically, but also socially by her weight. When she loses weight she always regains it and she always gains back more weight than she has lost. At one time, she lost 100 pounds and gained the weight back within a year. She has tried numerous commercial weight loss programs including Weight Watcher's for four months in 1992 with 15-pound weight loss, RS for two months in 1990 with six-pound weight loss, Slim Fast for six weeks in 2004 with eight-pound weight loss, an exercise program for two months in 2007 with a five-pound weight loss, Atkin's Diet for three months in 2008 with a ten-pound weight loss, and Dexatrim for one month in 2005 with a five-pound weight loss. She has also tried numerous fat reduction or fad diets. She was on Redux for nine months with a 100-pound weight loss.

PAST MEDICAL HISTORY: She has a history of hypertension and shortness of breath.

PAST SURGICAL HISTORY: Pertinent for cholecystectomy.

PSYCHOLOGICAL HISTORY: Negative.

SOCIAL HISTORY: She is single. She drinks alcohol once a week. She does not smoke.

FAMILY HISTORY: Pertinent for obesity and hypertension.

MEDICATIONS: Include Topamax 100 mg twice daily, Zoloft 100 mg twice daily, Abilify 5 mg daily, Motrin 800 mg daily, and a multivitamin.

ALLERGIES: She has no known drug allergies.

REVIEW OF SYSTEMS: Negative.

PHYSICAL EXAM: This is a pleasant female in no acute distress. Alert and oriented x 3. HEENT: Normocephalic, atraumatic. Extraocular muscles intact, nonicteric sclerae. Chest is clear to auscultation bilaterally. Cardiovascular is normal sinus rhythm. Abdomen is obese, soft, nontender and nondistended. Extremities show no edema, clubbing or cyanosis.

ASSESSMENT/PLAN: This is a 34-year-old female with a BMI of 43 who is interested in surgical weight via the gastric bypass as opposed to Lap-Band. Olivia Smith will be asking for a letter of medical necessity from Dr. Andrew Johnson. She will also see my nutritionist and social worker and have an upper endoscopy. Once this is completed, we will submit her to her insurance company for approval."""

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [8]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data)

,summary
0,"This is a clinical note about a 34-year-old woman who is interested in having weight loss surgery. She has been overweight for over 20 years and wants to have more energy and improve her self-image. She has tried many diets and weight loss programs, but has not been successful in keeping the weight off. She has a history of hypertension and shortness of breath, but is not allergic to any medications. She will have an upper endoscopy and will be contacted by a nutritionist and social worker. The plan is to have her weight loss surgery through the gastric bypass, rather than Lap-Band."


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [9]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data)

,summary
0,"This is a clinical note about a 38-year-old woman who is considering bariatric surgery for weight loss. She has been struggling with obesity for more than 25 years and wants to improve her physical stamina and self-perception. She has tried various diets and weight loss programs, including Jenny Craig, Optifast, Herbalife, a gym regimen, South Beach Diet, and Alli. She has a history of diabetes and sleep apnea, and has had appendectomy in the past. She takes Metformin, Amlodipine, Lipitor, Metoprolol, and a daily multivitamin. She will schedule appointments with a nutritionist and social worker and undergo an upper endoscopy. She will be approved for bariatric surgery and will be requesting a letter of medical necessity from her doctor."
1,"This is a clinical note about a 45-year-old man who had severe abdominal pain in the right lower quadrant of his abdomen. He had an emergency appendectomy, which was performed to remove the appendix. The procedure was successful and the patient recovered well after two days. The patient was diagnosed with acute appendicitis, which means that the appendix was inflamed and causing pain. The patient underwent an emergency appendectomy, which means that the appendix was removed. The procedure was uneventful, and the patient recovered well after two days."


### JSON Lines


In [10]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [11]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"summary": "This is a clinical note about a 38-year-old woman who is considering bariatric surgery for weight loss. She has been struggling with obesity for more than 25 years and wants to improve her physical stamina and self-perception. She has tried various diets and weight loss programs, including Jenny Craig, Optifast, Herbalife, a gym regimen, South Beach Diet, and Alli. She has a history of diabetes and sleep apnea, and has had appendectomy in the past. She takes Metformin, Amlodipine, Lipitor, Metoprolol, and a daily multivitamin. She will schedule appointments with a nutritionist and social worker and undergo an upper endoscopy. She will be approved for bariatric surgery and will be requesting a letter of medical necessity from her doctor."}
{"summary": "This is a clinical note about a 45-year-old man who had severe abdominal pain in the right lower quadrant of his abdomen. He had an emergency appendectomy, which was performed to remove the appendix. The procedure was success

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )



In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [ ]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [ ]:
process_s3_json_output_and_save(validation_json_file_name)

,summary
0,"This is a clinical note about a 38-year-old woman who is considering bariatric surgery for weight loss. She has been struggling with obesity for more than 25 years and wants to improve her physical stamina and self-perception. She has tried various diets and weight loss programs, including Jenny Craig, Optifast, Herbalife, a gym regimen, South Beach Diet, and Alli. She has a history of diabetes and sleep apnea, and has had appendectomy in the past. She takes Metformin, Amlodipine, Lipitor, Metoprolol, and a daily multivitamin. She will schedule appointments with a nutritionist and social worker and undergo an upper endoscopy. She will be approved for bariatric surgery and will be requesting a letter of medical necessity from her doctor."
1,"This is a clinical note about a 45-year-old man who had severe abdominal pain in the right lower quadrant of his abdomen. He had an emergency appendectomy, which was performed to remove the appendix. The procedure was successful and the patient recovered well after two days. The patient was diagnosed with acute appendicitis, which means that the appendix was inflamed and causing pain. The patient underwent an emergency appendectomy, which means that the appendix was removed. The procedure was uneventful, and the patient recovered well after two days."


### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [ ]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [ ]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"summary": "This is a clinical note about a 38-year-old woman who is considering bariatric surgery for weight loss. She has been struggling with obesity for more than 25 years and wants to improve her physical stamina and self-perception. She has tried various diets and weight loss programs, including Jenny Craig, Optifast, Herbalife, a gym regimen, South Beach Diet, and Alli. She has a history of diabetes and sleep apnea, and has had appendectomy in the past. She takes Metformin, Amlodipine, Lipitor, Metoprolol, and a daily multivitamin. She will schedule appointments with a nutritionist and social worker and undergo an upper endoscopy. She will be approved for bariatric surgery and will be requesting a letter of medical necessity from her doctor."}
{"summary": "This is a clinical note about a 45-year-old man who had severe abdominal pain in the right lower quadrant of his abdomen. He had an emergency appendectomy, which was performed to remove the appendix. The procedure was success

In [ ]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-summarize-clinical-laymen-onnx-pipel-2024-05-02-12-44-03-861


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

